# Анализ базы данных конкурента

Задачи:
- выяснить количество книг, выпущенных после 01-01-2000,
- узнать, какие книги наиболее высоко оцениваются,
- выяснить самое популярное издательство, выпускающее книги,
- узнать наиболее высоко оцениваемого автора,
- рассчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок


## 1. Изучение информации в таблицах

In [1]:
import pandas as pd 
from sqlalchemy import create_engine
from sqlalchemy import table, column

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имяпользователя
             'pwd': 'Sdf4$2;d-d30pp', #пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port': 6432,# портподключения
             'db': 'data-analyst-final-project-db'} # названиебазыданных


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],  
                                                         db_config['pwd'],  
                                                         db_config['host'],  
                                                         db_config['port'],  
                                                         db_config['db'])
# сохраняемконнектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [3]:
#выводим первые 5 строк с каждой таблицы
query = '''
           SELECT *
           FROM books
        '''

books = pd.io.sql.read_sql(query, con = engine)
display(books.head(5))

query = '''
           SELECT *
           FROM authors
        '''

authors = pd.io.sql.read_sql(query, con = engine)
display(authors.head(5))

query = '''
           SELECT *
           FROM publishers
        '''

publishers = pd.io.sql.read_sql(query, con = engine)
display(publishers.head(5))

query = '''
           SELECT *
           FROM ratings
        '''

ratings = pd.io.sql.read_sql(query, con = engine)
display(ratings.head(5))

query = '''
           SELECT *
           FROM reviews
        '''

reviews = pd.io.sql.read_sql(query, con = engine)
display(reviews.head(5))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 2. Расчет количества книг, выпущенных после 01-01-2000 года

In [4]:
query = '''
           SELECT
               COUNT(book_id) AS count_book
           FROM
               books
           WHERE
               CAST(publication_date AS date) > '2000-01-01' 
        '''

count_books = pd.io.sql.read_sql(query, con = engine)
display(count_books)


,count_book
0,819


После 01 января 2000 года всего выпущено 819 книг

## 3. Расчет количества обзоров и средней оценки

In [5]:

query = '''
           SELECT 
                books.book_id AS book_id,
                books.title AS title,
                COUNT(reviews.review_id) AS review_count,
                AVG(ratings.rating) AS average_rating
           FROM
                books
           INNER JOIN reviews ON reviews.book_id = books.book_id
           INNER JOIN ratings ON ratings.book_id = books.book_id
           GROUP BY
                books.book_id
           ORDER BY
                average_rating DESC,
                review_count DESC
                
        '''

reviews_rating_books = pd.io.sql.read_sql(query, con = engine)
display(reviews_rating_books)


,book_id,title,review_count,average_rating
0,17,A Dirty Job (Grim Reaper #1),16,5.00
1,553,School's Out—Forever (Maximum Ride #2),12,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,9,5.00
3,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
4,229,Evil Under the Sun (Hercule Poirot #24),4,5.00
...,...,...,...,...
989,915,The World Is Flat: A Brief History of the Twen...,12,2.25
990,202,Drowning Ruth,9,2.00
991,371,Junky,4,2.00
992,316,His Excellency: George Washington,4,2.00


Для книги с большинством обзоров, характерна средняя оценка 3,7. Самое большое количество обзоров со средним рейтингом 5, составляет 16-12 обзоров 

## 4. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''
           SELECT 
                publishers.publisher AS name_publisher,
                COUNT(books.book_id) AS books_count
           FROM
                books
           INNER JOIN publishers ON publishers.publisher_id = books.publisher_id  
           WHERE
                books.num_pages > 50
           GROUP BY
                publishers.publisher
           ORDER BY
                books_count DESC
        '''

pages_books = pd.io.sql.read_sql(query, con = engine)
display(pages_books)


,name_publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Наибольшее число книг с количеством страниц больше 50 выпустило издательство: Penguin Books (42 книги)

## 5. Определение автора с самой высокой средней оценкой книг 

In [7]:
query = '''
           SELECT
                   name_author,
                   rating_count,
                   average_rating
           FROM 
                   (SELECT
                           authors.author AS name_author,
                           COUNT(ratings.rating) AS rating_count,
                           AVG(ratings.rating) AS average_rating
                    FROM
                           books
                    INNER JOIN ratings ON ratings.book_id = books.book_id
                    INNER JOIN authors ON authors.author_id = books.author_id
                    GROUP BY
                           books.author_id,
                           authors.author) AS S
           WHERE rating_count > 50
           GROUP BY
                 name_author,
                 rating_count,
                 average_rating
           ORDER BY
                 average_rating DESC
                
        '''

author_rating = pd.io.sql.read_sql(query, con = engine)
display(author_rating)

,name_author,rating_count,average_rating
0,J.K. Rowling/Mary GrandPré,312,4.288462
1,Agatha Christie,53,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,53,4.264151
3,J.R.R. Tolkien,166,4.240964
4,Roald Dahl/Quentin Blake,62,4.209677
5,Louisa May Alcott,54,4.203704
6,Rick Riordan,84,4.130952
7,Arthur Golden,56,4.107143
8,Stephen King,106,4.009434
9,John Grisham,70,3.971429


Самую высокую среднюю оценку получили авторы J.K. Rowling/Mary GrandPré и Agatha Christie

## 5. Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [8]:
query = '''
            SELECT  
                   AVG(review_count) AS average_review_count
            
            FROM
                   (SELECT
                         username,
                         COUNT(review_id) AS review_count
                   FROM
                         reviews           
                   WHERE username IN
                                           (SELECT 
                                                 username
                                           FROM
                                                 ratings        
                                           GROUP BY
                                                 username
                                           HAVING
                                                 COUNT(rating) > 50)

                   GROUP BY
                         username) AS Sub
           
           
        '''

mean_reviews = pd.io.sql.read_sql(query, con = engine)
display(mean_reviews)


,average_review_count
0,24.333333


В среднем пользователи, которые оцени 50 и более книг, пишут около 24 обзоров